In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Run dependencies
%run transformation.ipynb

500 columns processed...
1000 columns processed...
1500 columns processed...
500 columns processed...
1000 columns processed...
1500 columns processed...


# Loading

In [14]:
# *******************************************LOADING*******************************************
import psycopg2
%run config.ipynb
engine=create_engine(f"postgresql://{connection_string}")

In [15]:
df_sales_raw.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1938,d_1939,d_1940,d_1941,id,cat_id,dept_id,item_id,state_id,store_id
0,3,4,2,1,0,3,2,2,4,0,...,0,3,2,4,2081,2,5,1340,6,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1716,1,3,1114,55,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1921,1,4,1233,55,8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1598,1,3,1045,55,9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1949,1,4,1253,48,6


In [16]:
holiday_df.to_sql(name="holiday", con=engine, if_exists='append', index=True)

walmart.to_sql(name="walmart", con=engine, if_exists='append', index=True)

stock.to_sql(name="stock", con=engine, if_exists='append', index=True)

state_id.to_sql(name="state_id", con=engine, if_exists='append', index=True)

store.to_sql(name="store", con=engine, if_exists='append', index=True)

marketShare_final.to_sql(name="marketshare", con=engine, if_exists='append', index=True)

In [17]:
# Add primary keys
engine.execute('ALTER TABLE "store" ADD PRIMARY KEY (id)')


In [18]:
df_sales.to_csv(('Resources/clean/d3_sales.csv'),index=False)
df_price_changes.to_csv(('Resources/clean/d3_price_changes.csv'),index=False)

df_sales_categories.to_csv(('Resources/clean/d3_categories.csv'),index=False)
df_sales_departments.to_csv(('Resources/clean/d3_departments.csv'),index=False)
df_sales_stores.to_csv(('Resources/clean/d3_stores.csv'),index=False)
df_sales_states.to_csv(('Resources/clean/d3_states.csv'),index=False)
df_sales_items.to_csv(('Resources/clean/d3_items.csv'),index=False)

df_ecomm.to_csv(('Resources/clean/d7_ecomm.csv'),index=False)

SyntaxError: unmatched ')' (<ipython-input-18-915b9fe45efb>, line 1)

In [ ]:
# this code loops through the folder of cleaned .csv files and loads them to PostgreSQL
# this is over 20 faster than using sqlalchemy and df.to_sql for long tables

# files are read to memory using StringIO in the io package
# 'copy [table] from stdin' in PostgreSQL, which directly from memory on the local computer

folder_name = os.path.join('Resources/clean')

conn_host = 'otto.db.elephantsql.com'
conn_dbname = 'ofiglsqd'
conn_user = 'ofiglsqd'

conn_pass = getpass.getpass(prompt='password: ')

# loop through .csv files in the output folder
for file in os.listdir(folder_name):

    print('\n\n' + str(datetime.utcnow()) + ' ' + str(file) + ' to be loaded')

    print(str(datetime.utcnow()) + ' reading file to dataframe...')
    
    # read .csv file into dataframe
    df = pd.read_csv(os.path.join(folder_name, file), na_values=['nan','NA','NaN'])
    
    print(str(datetime.utcnow()) + ' completed')
    
    print(df.info(memory_usage='deep'))
    
    # 
    with psycopg2.connect(host=conn_host, dbname=conn_dbname, user=conn_user, password=conn_pass) as conn:
        conn.autocommit = True

        table_name = file.split('.csv')[0].lower().replace('-','_')

        output = io.StringIO()

        print(str(datetime.utcnow()) + ' reading file to memory using StringIO...')

        df.to_csv(output, sep='|', header=False, index=False)
        output.seek(0)

        print(str(datetime.utcnow()) + ' completed')

        print(str(datetime.utcnow()) + ' generating the create table statement...')
        
        qry = pd.io.sql.get_schema(df, table_name, con=conn)

        qry = qry.replace('CREATE TABLE', 'CREATE TABLE IF NOT EXISTS')

        for key in df.columns:
            if pd.api.types.infer_dtype(df[key], skipna=True) == 'boolean':
                start = qry.find(key)
                end = start + qry[start:].find(',')
                print(start, end)
                qry = qry[:start] + key + '" BOOLEAN' + qry[end:]
        try:
            with conn.cursor() as cur:
                print(str(datetime.utcnow()) + ' completed')
                print(qry)
                
                print(str(datetime.utcnow()) + ' executing the create table statement...')
                cur.execute(qry)
                print(str(datetime.utcnow()) + ' completed')
                
                print(str(datetime.utcnow()) + ' loading table to database...')
                cur.copy_expert("""COPY %s FROM STDIN WITH (FORMAT csv, DELIMITER '|', QUOTE '"')""" % table_name, output)
                print(str(datetime.utcnow()) + ' completed')

        except Exception as e:
            print('Error:\n' + str(e))


In [ ]:
# Add financial data tables
revenue_df.to_sql(name='revenue', con=engine, if_exists='append', index=True)
netincome_df.to_sql(name='net_income', con=engine, if_exists='append', index=True)
earnings_df.to_sql(name='earnings', con=engine, if_exists='append', index=True)

In [ ]:
# Check tables in database
engine.table_names()

In [ ]:
# Close connection
engine.dispose()